# Continuous $paO_2$ Prediction and Postoperative Complications in

Neurosurgical Patients

AUC Calculation

Andrea S. Gutmann  
2025-12-17

## 1 Loading required libraries and data

In [1]:
# ======================
# Standard library
# ======================
import math
import os
import pickle
import platform
import pprint
import random
import sys
import time
from collections import Counter
from datetime import datetime
from pathlib import Path
from time import perf_counter

# ======================
# Third-party libraries
# ======================
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml
from scipy import stats
from sklearn.linear_model import SGDRegressor

# ======================
# Local / application
# ======================
import session_info

pp = pprint.PrettyPrinter(indent=4)
np.random.seed(42)


with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

with open("data/out/data_ready.pickle", 'rb') as d:
    data = pickle.load(d)

with open(Path(config.get('pickle').get("pre_abg")), 'rb') as f:
    pre_df = pickle.load(f)

with open(Path(config.get('pickle').get("post_abg")), 'rb') as f:
    post_df = pickle.load(f)

Functions

In [2]:
def get_auc(values: list, ) -> float:
    timepoints, values_segment = zip(*values)
    auc = np.trapezoid(values_segment, timepoints)
    return auc

def current_stats(df, p=False):
    """
    Calculate and optionally print summary statistics about a dataset.

    Parameters
    ----------
    df : pandas.DataFrame
        The input dataframe containing patient observations.
    p : bool, optional (default=False)
        If True, print a formatted summary message.

    Returns
    -------
    total : int
        Total number of rows (observations) in the dataframe.
    patients : int
        Number of unique patients (based on `identifier` column).
    """
    
    # Total number of rows (observations) in the dataframe
    total = len(df)
    
    # Number of unique patients, assuming 'identifier' uniquely marks each patient
    patients = len(df.identifier.unique())
    
    # Return the total number of rows and unique patients
    return total, patients

## 2 Combine both datasets

In [3]:
# Perform the merge
data = data.merge(
    pre_df[['idx', 'identifier', 'paO2_predicted']],
    on=['idx', 'identifier'],
    how='left'
)

# Perform the merge
data = data.merge(
    post_df[['idx', 'identifier', 'paO2_predicted']],
    on=['idx', 'identifier'],
    how='left'
)

# Create combined column from paO2_predicted_x and paO2_predicted_y
data['paO2_combined'] = data['paO2_predicted_x'].fillna(data['paO2_predicted_y'])

# Fill remaining missing values with paO2_measured
data['paO2_combined'] = data['paO2_combined'].fillna(data['paO2_measured'])

# Optional: drop the now-unneeded columns
data.drop(columns=['paO2_predicted_x', 'paO2_predicted_y', 'paO2_measured'], inplace=True)

data = data.sort_values(by=['identifier', 'idx'])

with open(Path(config.get('pickle').get('paO2_imputed_data')),'wb') as f:
    pickle.dump(data, f)

/var/folders/ff/zlqg1pks12q23nf7rf3m7j3r0000gn/T/ipykernel_82081/3535474277.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['paO2_combined'] = data['paO2_predicted_x'].fillna(data['paO2_predicted_y'])

## 3 Data cleaning

Remove negative predictions

In [4]:
print(current_stats(data))
print(data.paO2_combined.describe())
print(f"Removing {len(data.loc[data.paO2_combined <=0,:])} with negative paO\u2082 predictions.")
data = data.loc[data.paO2_combined >0,:]
print(current_stats(data))
print(f"Removing {len(data.loc[data.paO2_combined >1000,:])} with paO\u2082 predictions > 1000 mmHg.")
data = data.loc[data.paO2_combined <1000,:]
print(current_stats(data))

(326933, 5122)
count    326933.000000
mean        217.881708
std          80.621355
min         -23.531946
25%         164.420437
50%         196.001233
75%         242.686330
max         678.321534
Name: paO2_combined, dtype: float64
Removing 26 with negative paO₂ predictions.
(326907, 5122)
Removing 0 with paO₂ predictions > 1000 mmHg.
(326907, 5122)

Remove surgeries where to much data were excluded

In [5]:
min_mv_time = min(data.mv_time)
min_num_rows = int(min_mv_time/5)

threshold = int(2/3*min_num_rows)

print(current_stats(data))

mv_time_df = data[['identifier', 'mv_time']].drop_duplicates()
mv_time_df['mv_time_5'] = mv_time_df['mv_time'].apply(lambda x: int(x/5))

counter_dict = Counter(data.identifier)
mv_time_df['ident_counter'] = mv_time_df['identifier'].apply(lambda x: counter_dict.get(x))
mv_time_df['keep'] = mv_time_df.apply(lambda x: x.mv_time_5*(2/3)<=x.ident_counter, axis=1)

print(f"Remove surgeries with less than 66 % observations of their mechanical ventilation time.")
data = data[data.identifier.isin(mv_time_df.loc[mv_time_df['keep'],"identifier"])]
print(current_stats(data))

(326907, 5122)
Remove surgeries with less than 66 % observations of their mechanical ventilation time.
(324593, 5020)

Remove those without any predicted value

In [6]:
qm_dict = {}
for ident in data.identifier.unique():
    qm_dict[ident] = {
        'measured': len(data.loc[(data.identifier==ident) & (data.pao2_type_measured==True),:]),
        'predicted': len(data.loc[(data.identifier==ident) & (data.pao2_type_measured==False),:])
    }

print("Order by measured paO2")
print(sorted(qm_dict.items(), key = lambda x: x[1].get('measured')))

print("Order by predicted paO2")
print(sorted(qm_dict.items(), key = lambda x: x[1].get('predicted')))

# remove those without any predicted value
identifier_to_remove = [t[0] for t in list(filter(lambda x: x[1].get('predicted')== 0, qm_dict.items()))]

data = data[~data.identifier.isin(identifier_to_remove)]
print(current_stats(data))

Order by measured paO2
[(np.int64(376313), {'measured': 1, 'predicted': 58}), (np.int64(388994), {'measured': 1, 'predicted': 47}), (np.int64(389555), {'measured': 1, 'predicted': 42}), (np.int64(395957), {'measured': 1, 'predicted': 38}), (np.int64(397933), {'measured': 1, 'predicted': 33}), (np.int64(400293), {'measured': 1, 'predicted': 16}), (np.int64(404654), {'measured': 1, 'predicted': 59}), (np.int64(406001), {'measured': 1, 'predicted': 31}), (np.int64(419451), {'measured': 1, 'predicted': 11}), (np.int64(421703), {'measured': 1, 'predicted': 38}), (np.int64(422118), {'measured': 1, 'predicted': 14}), (np.int64(429073), {'measured': 1, 'predicted': 48}), (np.int64(432673), {'measured': 1, 'predicted': 35}), (np.int64(433815), {'measured': 1, 'predicted': 42}), (np.int64(436484), {'measured': 1, 'predicted': 25}), (np.int64(437833), {'measured': 1, 'predicted': 37}), (np.int64(440653), {'measured': 1, 'predicted': 24}), (np.int64(453796), {'measured': 1, 'predicted': 34}), (np.

Add p/F ratio

In [7]:
data["horowitz"] = data["last_horowitz"]
data.loc[data.horowitz.isnull(), "horowitz"] = data.loc[data.horowitz.isnull(), "first_horowitz"]
data.loc[data.horowitz.isnull(), "horowitz"] = data[data.horowitz.isnull()].apply(lambda x: x["paO2_combined"]/x["fio2"], axis=1)

## 4 Calculate perioperative AUC ($paO_2$ and p/F ratio)

In [8]:
data["norm_auc_paO2"] = None
data["norm_auc_pf"] = None
data["max_paO2"] = None

import warnings

warnings.filterwarnings("error", category=RuntimeWarning)

for ident in data.identifier.unique():
    normalized_paO2 = None
    highest_paO2 = None
    normalized_horowitz = None

    data_list_paO2_raw = data.loc[data.identifier==ident,["idx", "paO2_combined"]].values
    data_list_pf_raw = data.loc[data.identifier==ident,["idx", "horowitz"]].values
    min_idx = min([int(e[0]) for e in data_list_paO2_raw])
    max_idx = max([int(e[0]) for e in data_list_paO2_raw])
    data_list_paO2 = [[e[0]*5, e[1]] for e in data_list_paO2_raw]
    data_list_pf = [[e[0]*5, e[1]] for e in data_list_pf_raw]

    paO2_auc = get_auc(data_list_paO2)
    pf_auc = get_auc(data_list_pf)

    try:
        normalized_paO2 = paO2_auc/((max_idx-min_idx)*5)
        highest_paO2 = max([e[1] for e in data_list_paO2])
        normalized_horowitz = pf_auc/((max_idx-min_idx)*5)
    except RuntimeWarning as w:
        print(data_list_paO2)
        print(data_list_pf)
        print(paO2_auc, pf_auc, max_idx, min_idx)

    data.loc[data.identifier==ident,["norm_auc_paO2", "max_paO2", "norm_auc_pf", "auc_paO2"]] = normalized_paO2, highest_paO2, normalized_horowitz, paO2_auc

data = data.astype({"norm_auc_paO2": float, "norm_auc_pf": float, "max_paO2": float, "auc_paO2": float})

with open(Path(config.get('pickle').get('analysis')), 'wb') as f:
    pickle.dump(data, f)